# Deep Learning Models for stock prediction - LSTM

# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

## Import data

The data was downloaded from Bloomberg on:
* Exchange rate of Vietnam with its major trading partners: the China and the US
* Precious metal spot price and future price: Gold, Silver, Palladium, Platinum
* Global Stock Indices: Hang Seng Index, Nasdaq 100, Nasdaq Composite, Nikkei 225, SP500, DOJI, Shanghai Shenzhen CSI3000, Shanghai Shenzhen Composite and Singapore Stock Index
* Volatility stock index: VIX Index

The data will be imported from previous EDA session, which has been cleaned.

In [2]:
# Import data
data = pd.read_csv('data.csv')

# Transfer date column to date time
import datetime
data['Date'] = pd.to_datetime(data['Date'], format = '%m/%d/%Y')

# Turn date into index
data.set_index('Date', inplace = True)

In [3]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Generates dataset windows

    Args:
      series (array of float) - contains the values of the time series
      window_size (int) - the number of time steps to include in the feature
      batch_size (int) - the batch size
      shuffle_buffer(int) - buffer size to use for the shuffle method

    Returns:
      dataset (TF Dataset) - TF Dataset containing time windows
    """

    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)

    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Create tuples with features and labels
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Shuffle the windows
    dataset = dataset.shuffle(shuffle_buffer)

    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)

    return dataset

# Deep Learning model - LSTM

We will build two versions of the model
- Univariate LSTM prediction (using the target VN-Index price only)
- Multivariate LSTM prediction (combine with global stock indices and commodity price)

## Prepare model - Multivariate

In [4]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [8]:
# Setup batch size, epochs
epochs = 120
window_size = 10
batch_size = 64
shuffle_buffer_size = 1000

In [9]:
# Scaling the data
scaler = MinMaxScaler()

data_scaled = scaler.fit_transform(data)

In [12]:
# Split the dataset
train_time = int(round(len(data) * 0.8))

X_train = data_scaled[:train_time]
X_test = data_scaled[train_time:]

# Define dataset
dataset = windowed_dataset(X_train, window_size, batch_size, shuffle_buffer_size)

In [15]:
# Build model
model = Sequential([
    LSTM(64,  input_shape = (X_train.shape[1], X_train.shape[2]), return_sequences= True),
    LSTM(64, return_state= False),
    Dropout(0.1),
    Dense(1)
])

# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10 ** (epoch / 20)
)

# Initialize the optimizer
optimizer = tf.keras.optimizers.Adam()

# Compile model
model.compile(optimizer = optimizer, loss = 'mean_absolute_error')

# Check the model
model.summary()

IndexError: tuple index out of range

In [81]:
# Train the model
history = model.fit(trainX, trainy, batch_size = batch_size, epochs = epochs, validation_data= (testX, testy))

Epoch 1/120
52/52 [==============================] - 7s 54ms/step - loss: 0.0556 - val_loss: 0.1281
Epoch 2/120
52/52 [==============================] - 2s 40ms/step - loss: 0.0191 - val_loss: 0.0845
Epoch 3/120
52/52 [==============================] - 2s 41ms/step - loss: 0.0175 - val_loss: 0.1090
Epoch 4/120
52/52 [==============================] - 2s 40ms/step - loss: 0.0186 - val_loss: 0.0946
Epoch 5/120
52/52 [==============================] - 2s 42ms/step - loss: 0.0176 - val_loss: 0.0531
Epoch 6/120
52/52 [==============================] - 2s 40ms/step - loss: 0.0166 - val_loss: 0.0681
Epoch 7/120
52/52 [==============================] - 2s 40ms/step - loss: 0.0161 - val_loss: 0.0381
Epoch 8/120
52/52 [==============================] - 2s 39ms/step - loss: 0.0158 - val_loss: 0.0500
Epoch 9/120
52/52 [==============================] - 2s 40ms/step - loss: 0.0150 - val_loss: 0.0370
Epoch 10/120
52/52 [==============================] - 2s 39ms/step - loss: 0.0151 - val_loss: 0.0387

In [88]:
# Make predictions
predictions = model.predict(testX)

# Refit the scaler
scaler.fit(data['index_vni'].values.reshape(-1,1))

# Inverse scaling
predictions_scaled = scaler.inverse_transform(predictions)

 1/25 [>.............................] - ETA: 0s

25/25 [==============================] - 0s 12ms/step


In [90]:
len(predictions_scaled)

799